In [1]:
import pandas as pd
import csv
import os
import requests
import urllib3
import csv
import time
import re
import numpy as np
# from random import randint
# from pandas import json_normalize
import censusgeocode as cg
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import shape
from datetime import datetime
os.chdir('C:/Users/jhuo/Documents/20220421_WABE_SB202/')

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
traveltimemaster5 = pd.read_csv('20220530_voter_data_all_vars.csv')

In [4]:
def race_recode(x):
    if x == 'WH':
        return(1)
    if x == ('U'):
        return(0)
    if x == ('OT'):
        return(0)
    else:
        return (-1)

In [5]:
traveltimemaster5['white'] = [race_recode(x) for x in traveltimemaster5.RACE]

In [6]:
traveltimemaster5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7528561 entries, 0 to 7528560
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   Unnamed: 0_x             int64  
 2   REGISTRATION_NUMBER      int64  
 3   BIRTHDATE_x              int64  
 4   RACE                     object 
 5   st_address               object 
 6   RESIDENCE_CITY           object 
 7   RESIDENCE_STATE          object 
 8   ZIP5                     int64  
 9   lat                      float64
 10  long                     float64
 11  min_driving_2020         float64
 12  min_driving_2022         float64
 13  driving_band_diff        float64
 14  BIRTHDATE_y              int64  
 15  min_transit_2020         float64
 16  min_transit_2022         float64
 17  transit_band_diff        float64
 18  urban                    bool   
 19  TRACTCE                  float64
 20  GEOID_left               float64
 21  NAME    

In [7]:
traveltimemaster5

,Unnamed: 0,Unnamed: 0_x,REGISTRATION_NUMBER,BIRTHDATE_x,RACE,st_address,RESIDENCE_CITY,RESIDENCE_STATE,ZIP5,lat,...,county_fips,index_right,GEOID_right,party,precint_keep,no_car,elderly,min_travel_2020_recode,min_travel_2022_recode,white
0,0,0,2,1937,U,237 MITCHELL RD,MAYSVILLE,GA,30558,34.267868,...,13011.0,33658.0,13011-01,R,True,0,1,1200.0,1200.0,0
1,1,1,3,1940,WH,467 HICKORY CREEK RD,MAYSVILLE,GA,30558,34.308254,...,13011.0,33658.0,13011-01,R,True,0,1,1200.0,1200.0,1
2,2,2,4,1954,WH,733 CARSON SEGARS RD,MAYSVILLE,GA,30558,34.295063,...,13011.0,33658.0,13011-01,R,True,0,1,600.0,600.0,1
3,3,3,7,1952,WH,11 W RIDGEWAY RD,MAYSVILLE,GA,30558,34.257618,...,13011.0,33658.0,13011-01,R,True,0,1,1200.0,1200.0,1
4,4,4,16,1937,U,262 HIGHWAY 52,MAYSVILLE,GA,30558,34.261142,...,13011.0,33658.0,13011-01,R,True,0,1,1200.0,1200.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528556,7528575,7545308,13307003,1984,WH,3301 MAPLE RD SE,LINDALE,GA,30147,34.194164,...,13115.0,34754.0,13115-WEST LINDALE (135),R,True,0,0,600.0,600.0,1
7528557,7528576,7545309,13307004,1995,WH,1 VINELAND DR NW,ROME,GA,30165,34.273735,...,13115.0,34741.0,13115-TOWN ROME (140),R,True,1,0,600.0,1200.0,1
7528558,7528577,7545310,13307005,1998,WH,39 THOMPSON RD SE,SILVER CREEK,GA,30173,34.170677,...,13115.0,34761.0,13115-EAST LINDALE (065),R,True,0,0,1200.0,600.0,1
7528559,7528578,7545311,13307006,2000,WH,12 HARBOUR RD NE,ROME,GA,30165,34.332336,...,13115.0,34752.0,13115-GLENWOOD (055),R,True,0,0,1200.0,1200.0,1


In [8]:
print('number of voters with 20 minute access who see their time go up')
len(traveltimemaster5[(traveltimemaster5.min_travel_time_2020 <=1200) &
                 (traveltimemaster5.min_travel_time_2022 <=1200) &
                 (traveltimemaster5.travel_diff < 0) ])

number of voters with 20 minute access who see their time go up


1856574

In [9]:
loseaccess = traveltimemaster5[(traveltimemaster5.min_travel_time_2020 <=1200) &
                 (traveltimemaster5.min_travel_time_2022 <=1200) &
                 (traveltimemaster5.travel_diff < 0) ]

In [10]:
len(loseaccess[loseaccess.urban == True])/len(loseaccess)

0.9325267939764319

In [11]:
len(loseaccess)/len(traveltimemaster5)

0.2466040987115599

In [12]:
len(traveltimemaster5[traveltimemaster5.urban == True])/len(traveltimemaster5)

0.7420277261484631

making crosstabs

In [13]:
def crosstab_traveltime(var_list):
    crosstab2020pct = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2020_recode.value_counts()/traveltimemaster5.groupby(var_list).REGISTRATION_NUMBER.count())
    crosstab2020pct.columns = ['pct_of_group']
    crosstab2020pct = crosstab2020pct.reset_index()

    crosstab2020cnt = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2020_recode.value_counts())
    crosstab2020cnt.columns = ['voter_count']
    crosstab2020cnt = crosstab2020cnt.reset_index()
    
    var_list.append('min_travel_2020_recode')
    newvarlist2020 = var_list.copy()
    var_list.remove('min_travel_2020_recode')
    crosstab2020merge = pd.merge(crosstab2020pct,crosstab2020cnt, on = newvarlist2020, how = 'outer')
    crosstab2020merge = crosstab2020merge.rename(columns = {'min_travel_2020_recode':'travel_time',
                                                           'pct_of_group':'pct_of_group_2020',
                                                           'voter_count':'voter_count_2020'})
    
    crosstab2022pct = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2022_recode.value_counts()/traveltimemaster5.groupby(var_list).REGISTRATION_NUMBER.count())
    crosstab2022pct.columns = ['pct_of_group']
    crosstab2022pct = crosstab2022pct.reset_index()

    crosstab2022cnt = pd.DataFrame(traveltimemaster5.groupby(var_list).min_travel_2022_recode.value_counts())
    crosstab2022cnt.columns = ['voter_count']
    crosstab2022cnt = crosstab2022cnt.reset_index()

    var_list.append('min_travel_2022_recode')
    newvarlist2022 = var_list.copy()
    var_list.remove('min_travel_2022_recode')
    crosstab2022merge = pd.merge(crosstab2022pct,crosstab2022cnt, on = newvarlist2022, how = 'outer')
    crosstab2022merge = crosstab2022merge.rename(columns = {'min_travel_2022_recode':'travel_time',
                                                           'pct_of_group':'pct_of_group_2022',
                                                           'voter_count':'voter_count_2022'})
    
    var_list.append('travel_time')
    mergevarlist = var_list.copy()
    crosstabmerge = pd.merge(crosstab2020merge, crosstab2022merge, on = mergevarlist, how = 'outer')
    crosstabmerge['pct_diff'] = crosstabmerge.pct_of_group_2020 - crosstabmerge.pct_of_group_2022
    return crosstabmerge

In [14]:
ctyct = crosstab_traveltime(['county_fips'])

In [15]:
ctyct.sort_values('pct_diff', ascending = False).head(20)

,county_fips,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
276,13245.0,600.0,0.765832,113105.0,0.190224,28094,0.575608
181,13163.0,600.0,0.845354,9588.0,0.295715,3354,0.549639
132,13121.0,600.0,0.954616,785311.0,0.461414,379580,0.493202
146,13135.0,600.0,0.863163,524764.0,0.394342,239742,0.468821
73,13067.0,600.0,0.938253,512603.0,0.488128,266683,0.450124
104,13097.0,600.0,0.748658,78097.0,0.300357,31332,0.448301
39,13029.0,1200.0,0.575887,19006.0,0.157834,5209,0.418053
279,13247.0,600.0,0.974431,65167.0,0.573441,38350,0.400990
120,13113.0,600.0,0.718119,67236.0,0.320065,29967,0.398054
66,13059.0,600.0,0.885659,70378.0,0.500919,39805,0.384740


the limitations on drop boxes are making it more difficult for those communities that rely on them most.

In [16]:
whitenonwhite = crosstab_traveltime(['white'])

In [17]:
whitenonwhite

,white,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,-1,600.0,0.737159,2018862,0.472808,1294882,0.264351
1,-1,1200.0,0.180447,494190,0.480717,1316541,-0.300270
2,-1,1800.0,0.020977,57451,0.043475,119066,-0.022498
3,0,600.0,0.628945,537557,0.401813,343428,0.227132
4,0,1200.0,0.254534,217549,0.517522,442324,-0.262988
5,0,1800.0,0.042096,35979,0.071519,61127,-0.029423
6,1,600.0,0.501028,1971624,0.357161,1405484,0.143867
7,1,1200.0,0.328276,1291819,0.532065,2093762,-0.203789
8,1,1800.0,0.060289,237245,0.096963,381563,-0.036674


In [18]:
racecty = crosstab_traveltime(['county_fips','white'])

In [19]:
racecty.head()

,county_fips,white,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,13001.0,-1,600.0,0.715154,1765.0,0.688006,1698.0,0.027147
1,13001.0,-1,1200.0,0.274716,678.0,0.301053,743.0,-0.026337
2,13001.0,-1,1800.0,0.010130,25.0,0.010535,26.0,-0.000405
3,13001.0,0,600.0,0.515679,444.0,0.491289,423.0,0.024390
4,13001.0,0,1200.0,0.416957,359.0,0.428571,369.0,-0.011614


In [20]:
racectypoc = racecty[racecty.white == -1][['county_fips','travel_time','voter_count_2020']]

In [21]:
racectypoc.head()

,county_fips,travel_time,voter_count_2020
0,13001.0,600.0,1765.0
1,13001.0,1200.0,678.0
2,13001.0,1800.0,25.0
9,13003.0,600.0,717.0
10,13003.0,1200.0,527.0


In [22]:
sum1 = pd.DataFrame(traveltimemaster5[traveltimemaster5.white == -1].min_travel_2020_recode.value_counts())

In [23]:
sum1 = sum1.reset_index()
sum1.columns = ['travel_time','target']

In [24]:
racectypoc1 = pd.merge(racectypoc, sum1, on = 'travel_time', how = 'left')

In [25]:
racectypoc1 = racectypoc1.rename(columns = {'voter_count_2020':'source'})

In [26]:
race = crosstab_traveltime(['white'])

In [27]:
racepoc = race[race.white == -1][['travel_time','voter_count_2020']]

In [28]:
# teasing this apart
len(traveltimemaster5[(traveltimemaster5.white == -1) & 
                 (traveltimemaster5.min_travel_2020_recode == 600)])/len(traveltimemaster5[traveltimemaster5.white == -1])

0.7371593508610822

In [29]:
poc600pct = pd.DataFrame(traveltimemaster5[(traveltimemaster5.white == -1) & 
                 (traveltimemaster5.min_travel_2020_recode == 600)].county_fips.value_counts()/len(traveltimemaster5[(traveltimemaster5.white == -1) & 
                 (traveltimemaster5.min_travel_2020_recode == 600)]))

In [30]:
poc600pct.county_fips.iloc[:4].sum()

0.5570796815235514

In [31]:
pocctypct = pd.DataFrame(traveltimemaster5[(traveltimemaster5.white == -1)].county_fips.value_counts()/len(traveltimemaster5[(traveltimemaster5.white == -1)]))

In [32]:
pocctypct.county_fips.iloc[:4]

13121.0    0.144528
13089.0    0.113356
13135.0    0.105945
13067.0    0.070878
Name: county_fips, dtype: float64

In [33]:
pocctypct.head(10)

,county_fips
13121.0,0.144528
13089.0,0.113356
13135.0,0.105945
13067.0,0.070878
13063.0,0.055873
13151.0,0.033776
13051.0,0.032485
13245.0,0.029949
13215.0,0.025301
13021.0,0.022150


In [34]:
0.264351/0.737159 	

0.35860784444061594

In [35]:
urbanwhitenonwhite = crosstab_traveltime(['white','urban'])

In [36]:
urbanwhitenonwhite.sort_values(['urban','white','travel_time'])

,white,urban,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
1,-1,False,600.0,0.271472,91003,0.275828,92463,-0.004355
0,-1,False,1200.0,0.423077,141824,0.564590,189262,-0.141513
2,-1,False,1800.0,0.073632,24683,0.146414,49081,-0.072782
7,0,False,600.0,0.213910,36444,0.201108,34263,0.012801
6,0,False,1200.0,0.488293,83191,0.585733,99792,-0.097440
8,0,False,1800.0,0.107870,18378,0.184709,31469,-0.076838
13,1,False,600.0,0.207417,297969,0.208645,299733,-0.001228
12,1,False,1200.0,0.492369,707322,0.587485,843963,-0.095116
14,1,False,1800.0,0.111165,159696,0.177826,255459,-0.066661
3,-1,True,600.0,0.802110,1927859,0.500281,1202419,0.301828


In [37]:
urbanct = crosstab_traveltime(['urban'])

In [38]:
urbanct.sort_values(['urban','travel_time'])

,urban,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
1,False,600.0,0.219043,425416,0.219580,426459,-0.000537
0,False,1200.0,0.480052,932337,0.583380,1133017,-0.103328
2,False,1800.0,0.104398,202757,0.173008,336009,-0.068610
3,True,600.0,0.734395,4102627,0.468519,2617335,0.265876
4,True,1200.0,0.191755,1071221,0.486827,2719610,-0.295072
5,True,1800.0,0.022898,127918,0.040410,225747,-0.017512


In [39]:
0.265876/0.734395 	

0.36203405524275084

In [40]:
nocarct = crosstab_traveltime(['no_car'])

In [41]:
nocarct

,no_car,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,0,600.0,0.570286,3863411,0.386832,2620602,0.183453
1,0,1200.0,0.289060,1958242,0.521194,3530839,-0.232134
2,0,1800.0,0.048373,327701,0.081640,553072,-0.033267
3,1,600.0,0.881425,664632,0.561231,423192,0.320194
4,1,1200.0,0.060097,45316,0.426750,321788,-0.366653
5,1,1800.0,0.003944,2974,0.011517,8684,-0.007573


In [42]:
incomect = crosstab_traveltime(['income_qt'])

In [43]:
incomect

,income_qt,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,1.0,600.0,0.682120,1279428,0.534047,1001693,0.148073
1,1.0,1200.0,0.168115,315328,0.406837,763089,-0.238721
2,1.0,1800.0,0.029472,55279,0.055194,103526,-0.025723
3,2.0,600.0,0.537289,1008417,0.392211,736127,0.145077
4,2.0,1200.0,0.303073,568827,0.504528,946930,-0.201455
5,2.0,1800.0,0.054099,101537,0.093069,174677,-0.038969
6,3.0,600.0,0.569848,1069106,0.341390,640491,0.228457
7,3.0,1200.0,0.313931,588975,0.579556,1087320,-0.265624
8,3.0,1800.0,0.042416,79578,0.072004,135088,-0.029588
9,4.0,600.0,0.616292,1154666,0.350552,656784,0.265740


In [44]:
agect = crosstab_traveltime(['elderly'])

In [45]:
agect

,elderly,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,0,600.0,0.611591,3637822,0.405037,2409211,0.206554
1,0,1200.0,0.261498,1555422,0.514163,3058312,-0.252666
2,0,1800.0,0.041743,248291,0.072076,428716,-0.030333
3,1,600.0,0.563278,890221,0.401526,634583,0.161752
4,1,1200.0,0.283553,448136,0.502595,794315,-0.219041
5,1,1800.0,0.052128,82384,0.084180,133040,-0.032052


In [46]:
partyct = crosstab_traveltime(['party'])

In [47]:
partyct.sort_values(['party','travel_time'])

,party,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,D,600.0,0.839271,3062087,0.498253,1817880,0.341018
1,D,1200.0,0.122513,446991,0.474520,1731289,-0.352006
2,D,1800.0,0.010866,39643,0.026578,96969,-0.015712
4,R,600.0,0.377712,1462776,0.316206,1224577,0.061507
3,R,1200.0,0.400987,1552911,0.546311,2115713,-0.145325
5,R,1800.0,0.075140,290996,0.119964,464587,-0.044824
7,tie,600.0,0.443386,3164,0.185398,1323,0.257988
6,tie,1200.0,0.511771,3652,0.787416,5619,-0.275645
8,tie,1800.0,0.004484,32,0.027186,194,-0.022702


In [48]:
cityct = crosstab_traveltime(['RESIDENCE_CITY'])

In [49]:
countyct = crosstab_traveltime(['county_fips'])

In [50]:
racect = crosstab_traveltime(['RACE'])

In [51]:
racect

,RACE,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,AI,600.0,0.651414,18308,0.409180,11500,0.242234
1,AI,1200.0,0.238427,6701,0.517559,14546,-0.279132
2,AI,1800.0,0.033802,950,0.063903,1796,-0.030101
3,AP,600.0,0.719363,144874,0.405622,81689,0.313741
4,AP,1200.0,0.208901,42071,0.537216,108191,-0.328315
5,AP,1800.0,0.035031,7055,0.053259,10726,-0.018228
6,BH,600.0,0.747011,1662342,0.486874,1083454,0.260136
7,BH,1200.0,0.169963,378222,0.469350,1044456,-0.299387
8,BH,1800.0,0.018197,40495,0.041345,92005,-0.023147
9,HP,600.0,0.681051,193338,0.416508,118239,0.264543


In [52]:
racecty = crosstab_traveltime(['county_fips','urban','white']).sort_values(['county_fips','urban','white','travel_time'])

In [53]:
traveltimemaster5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7528561 entries, 0 to 7528560
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   Unnamed: 0_x             int64  
 2   REGISTRATION_NUMBER      int64  
 3   BIRTHDATE_x              int64  
 4   RACE                     object 
 5   st_address               object 
 6   RESIDENCE_CITY           object 
 7   RESIDENCE_STATE          object 
 8   ZIP5                     int64  
 9   lat                      float64
 10  long                     float64
 11  min_driving_2020         float64
 12  min_driving_2022         float64
 13  driving_band_diff        float64
 14  BIRTHDATE_y              int64  
 15  min_transit_2020         float64
 16  min_transit_2022         float64
 17  transit_band_diff        float64
 18  urban                    bool   
 19  TRACTCE                  float64
 20  GEOID_left               float64
 21  NAME    

In [54]:
traveltimemaster5[traveltimemaster5.min_travel_2020_recode == 600].county_fips.value_counts()/len(traveltimemaster5[traveltimemaster5.min_travel_2020_recode == 600])

13121.0    0.173433
13089.0    0.120214
13135.0    0.115892
13067.0    0.113206
13063.0    0.037444
             ...   
13239.0    0.000278
13101.0    0.000271
13007.0    0.000185
13037.0    0.000174
13061.0    0.000040
Name: county_fips, Length: 121, dtype: float64

In [55]:
traveltimemaster5[traveltimemaster5.min_travel_2020_recode == 1200].county_fips.value_counts()/len(traveltimemaster5[traveltimemaster5.min_travel_2020_recode == 1200])

13057.0    0.058841
13117.0    0.058621
13151.0    0.054036
13135.0    0.041442
13139.0    0.034825
             ...   
13061.0    0.000459
13101.0    0.000420
13249.0    0.000341
13239.0    0.000226
13053.0    0.000131
Name: county_fips, Length: 121, dtype: float64

In [56]:
traveltimemaster5[traveltimemaster5.white == -1].county_fips.value_counts()/len(traveltimemaster5[traveltimemaster5.white == -1 ])

13121.0    0.144528
13089.0    0.113356
13135.0    0.105945
13067.0    0.070878
13063.0    0.055873
             ...   
13241.0    0.000144
13101.0    0.000107
13083.0    0.000106
13281.0    0.000074
13125.0    0.000047
Name: county_fips, Length: 159, dtype: float64

In [57]:
0.144528 + 0.113356 + 0.105945 + 0.070878

0.434707

In [58]:
incomtcqy = crosstab_traveltime(['county_fips','urban','income_qt']).sort_values(['county_fips','urban','income_qt','travel_time'])

In [59]:
def correlate(x,y):
    df = pd.DataFrame(traveltimemaster5.groupby(x)[y].value_counts()/traveltimemaster5.groupby(x).REGISTRATION_NUMBER.count())
    df.columns = ['pct_of_group']
    df = df.reset_index()
    countdf = pd.DataFrame(traveltimemaster5.groupby(x)[y].value_counts())
    countdf.columns = ['voter_count']
    countdf = countdf.reset_index()
    merge = pd.merge(df, countdf, on = [x,y], how = 'outer')
    return(merge.sort_values([x,y]))
    

In [60]:
correlate('urban','white')

,urban,white,pct_of_group,voter_count
1,False,-1,0.172602,335220
2,False,0,0.087722,170371
0,False,1,0.739676,1436569
4,True,-1,0.430239,2403485
5,True,0,0.122498,684325
3,True,1,0.447263,2498591


In [61]:
correlate('no_car','income_qt')

,no_car,income_qt,pct_of_group,voter_count
3,0,1.0,0.192336,1302983
2,0,2.0,0.263292,1783674
1,0,3.0,0.268359,1818006
0,0,4.0,0.273704,1854213
4,1,1.0,0.759481,572681
5,1,2.0,0.123586,93189
6,1,3.0,0.077078,58120
7,1,4.0,0.025670,19356


In [62]:
correlate('income_qt','no_car')

,income_qt,no_car,pct_of_group,voter_count
0,1.0,0,0.694678,1302983
1,1.0,1,0.305322,572681
2,2.0,0,0.950349,1783674
3,2.0,1,0.049651,93189
4,3.0,0,0.969021,1818006
5,3.0,1,0.030979,58120
6,4.0,0,0.989669,1854213
7,4.0,1,0.010331,19356


In [63]:
correlate('no_car','elderly')

,no_car,elderly,pct_of_group,voter_count
0,0,0,0.787791,5336903
1,0,1,0.212209,1437615
2,1,0,0.810602,611229
3,1,1,0.189398,142814


In [64]:
correlate('no_car','party').sort_values(['no_car','party'])

,no_car,party,pct_of_group,voter_count
1,0,D,0.447797,3033612
0,0,R,0.551473,3735963
2,0,tie,0.000701,4749
3,1,D,0.815464,614895
4,1,R,0.181370,136761
5,1,tie,0.003166,2387


In [65]:
nocarctycorr = correlate('no_car','county_fips').sort_values(['no_car','county_fips'])

In [66]:
nocarctycorr.to_csv('20220530_no_car_county.csv')

In [67]:
correlate('income_qt','elderly')

,income_qt,elderly,pct_of_group,voter_count
0,1.0,0,0.789079,1480047
1,1.0,1,0.210921,395617
2,2.0,0,0.785729,1474706
3,2.0,1,0.214271,402157
4,3.0,0,0.795956,1493314
5,3.0,1,0.204044,382812
6,4.0,0,0.789463,1479114
7,4.0,1,0.210537,394455


In [68]:
correlate('elderly','income_qt').sort_values(['elderly','income_qt'])

,elderly,income_qt,pct_of_group,voter_count
1,0,1.0,0.248826,1480047
3,0,2.0,0.247928,1474706
0,0,3.0,0.251056,1493314
2,0,4.0,0.248669,1479114
5,1,1.0,0.250323,395617
4,1,2.0,0.254461,402157
7,1,3.0,0.242220,382812
6,1,4.0,0.249587,394455


In [69]:
correlate('party','income_qt').sort_values(['party','income_qt'])

,party,income_qt,pct_of_group,voter_count
0,D,1.0,0.306590,1118596
2,D,2.0,0.221167,806928
1,D,3.0,0.261807,955204
3,D,4.0,0.205888,751185
7,R,1.0,0.194637,753776
5,R,2.0,0.276078,1069175
6,R,3.0,0.237094,918198
4,R,4.0,0.289717,1121994
8,tie,1.0,0.460482,3286
10,tie,2.0,0.103139,736


In [70]:
correlate('urban','no_car')

,urban,no_car,pct_of_group,voter_count
0,False,0,0.963842,1871935
1,False,1,0.036158,70225
2,True,0,0.877592,4902583
3,True,1,0.122408,683818


In [71]:
correlate('urban','income_qt')

,urban,income_qt,pct_of_group,voter_count
1,False,1.0,0.280502,544779
0,False,2.0,0.346397,672758
2,False,3.0,0.244055,473993
3,False,4.0,0.125654,244041
6,True,1.0,0.238237,1330885
7,True,2.0,0.215542,1204105
5,True,3.0,0.250990,1402133
4,True,4.0,0.291695,1629528


In [72]:
correlate('urban','elderly')

,urban,elderly,pct_of_group,voter_count
0,False,0,0.744142,1445243
1,False,1,0.255858,496917
2,True,0,0.806045,4502889
3,True,1,0.193955,1083512


In [73]:
correlate('urban','party')

,urban,party,pct_of_group,voter_count
1,False,D,0.106407,206659
0,False,R,0.892579,1733531
2,False,tie,0.000958,1861
3,True,D,0.616112,3441848
4,True,R,0.382929,2139193
5,True,tie,0.000944,5275


In [74]:
correlate('elderly','party')

,elderly,party,pct_of_group,voter_count
0,0,D,0.503090,2992444
1,0,R,0.495980,2950155
2,0,tie,0.000907,5394
4,1,D,0.415117,656063
3,1,R,0.583746,922569
5,1,tie,0.001102,1742


In [75]:
correlate('urban','RACE')

,urban,RACE,pct_of_group,voter_count
6,False,AI,0.002858,5551
5,False,AP,0.007642,14842
1,False,BH,0.142369,276503
3,False,HP,0.019733,38324
4,False,OT,0.010457,20310
2,False,U,0.077265,150061
0,False,WH,0.739676,1436569
13,True,AI,0.004037,22554
11,True,AP,0.033394,186550
8,True,BH,0.348851,1948823


In [76]:
correlate('no_car','RACE')

,no_car,RACE,pct_of_group,voter_count
6,0,AI,0.003696,25041
4,0,AP,0.027954,189375
1,0,BH,0.265948,1801669
3,0,HP,0.038627,261677
5,0,OT,0.019472,131915
2,0,U,0.094361,639251
0,0,WH,0.549942,3725590
13,1,AI,0.004063,3064
12,1,AP,0.015937,12017
7,1,BH,0.561847,423657


In [77]:
correlate('income_qt','RACE')

,income_qt,RACE,pct_of_group,voter_count
6,1.0,AI,0.003532,6624
5,1.0,AP,0.011775,22086
0,1.0,BH,0.453574,850752
3,1.0,HP,0.031714,59484
4,1.0,OT,0.016873,31649
2,1.0,U,0.082610,154949
1,1.0,WH,0.399922,750120
13,2.0,AI,0.003737,7014
12,2.0,AP,0.017424,32702
8,2.0,BH,0.298717,560650


In [78]:
correlate('elderly','RACE')

,elderly,RACE,pct_of_group,voter_count
6,0,AI,0.004379,26049
4,0,AP,0.028531,169708
1,0,BH,0.312891,1861116
3,0,HP,0.043956,261456
5,0,OT,0.022123,131593
2,0,U,0.098549,586185
0,0,WH,0.489570,2912025
13,1,AI,0.001301,2056
10,1,AP,0.020048,31684
8,1,BH,0.230450,364210


In [79]:
correlate('party','RACE')

,party,RACE,pct_of_group,voter_count
6,D,AI,0.004260,15543
4,D,AP,0.034656,126441
0,D,BH,0.473771,1728555
3,D,HP,0.043791,159770
5,D,OT,0.025482,92971
2,D,U,0.099785,364067
1,D,WH,0.318256,1161160
13,R,AI,0.003237,12537
11,R,AP,0.019298,74734
8,R,BH,0.127661,494397


In [80]:
urbanincome = crosstab_traveltime(['RACE','income_qt'])

In [81]:
urbanincome.sort_values(['RACE','income_qt','travel_time'])

,RACE,income_qt,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,AI,1.0,600.0,0.755435,5004,0.536081,3551,0.219354
1,AI,1.0,1200.0,0.146286,969,0.416214,2757,-0.269928
2,AI,1.0,1800.0,0.023098,153,0.044837,297,-0.021739
3,AI,2.0,600.0,0.591674,4150,0.413174,2898,0.178500
4,AI,2.0,1200.0,0.263473,1848,0.489877,3436,-0.226404
...,...,...,...,...,...,...,...,...
79,WH,3.0,1200.0,0.372422,370749,0.585711,583081,-0.213290
80,WH,3.0,1800.0,0.056042,55790,0.092657,92241,-0.036615
81,WH,4.0,600.0,0.592137,698279,0.339780,400686,0.252357
82,WH,4.0,1200.0,0.293539,346157,0.557388,657301,-0.263849


Forty states used drop boxes during the 2020 election

In [82]:
survey = pd.read_csv('MITU0031_OUTPUT2.tab', sep = '\t',low_memory=False)

In [83]:
surveyGA = survey[survey.inputstate == 13]

In [84]:
len(surveyGA[surveyGA.Q29 == 4])/len(surveyGA[surveyGA.Q28 != 99])

0.21476510067114093

In [85]:
len(survey[survey.Q29 == 4].inputstate.value_counts())

43

In [86]:
surveyGA.Q28.value_counts()

99    702
1     259
2      35
9       4
Name: Q28, dtype: int64

In [87]:
len(surveyGA[surveyGA.Q35 == 10])/len(surveyGA)

0.011

In [88]:
600000/7000000

0.08571428571428572

20220608 analyzing fulton county

In [89]:
traveltimemaster5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7528561 entries, 0 to 7528560
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   Unnamed: 0_x             int64  
 2   REGISTRATION_NUMBER      int64  
 3   BIRTHDATE_x              int64  
 4   RACE                     object 
 5   st_address               object 
 6   RESIDENCE_CITY           object 
 7   RESIDENCE_STATE          object 
 8   ZIP5                     int64  
 9   lat                      float64
 10  long                     float64
 11  min_driving_2020         float64
 12  min_driving_2022         float64
 13  driving_band_diff        float64
 14  BIRTHDATE_y              int64  
 15  min_transit_2020         float64
 16  min_transit_2022         float64
 17  transit_band_diff        float64
 18  urban                    bool   
 19  TRACTCE                  float64
 20  GEOID_left               float64
 21  NAME    

In [90]:
fulton = traveltimemaster5[traveltimemaster5.county_fips == 13121].copy()

In [91]:
pd.DataFrame(fulton.min_travel_2022_recode.value_counts()/len(fulton))

,min_travel_2022_recode
1200.0,0.529795
600.0,0.461414
1800.0,0.008791


In [92]:
def crosstab_traveltime_df(df,var_list):
    crosstab2020pct = pd.DataFrame(df.groupby(var_list).min_travel_2020_recode.value_counts()/df.groupby(var_list).REGISTRATION_NUMBER.count())
    crosstab2020pct.columns = ['pct_of_group']
    crosstab2020pct = crosstab2020pct.reset_index()

    crosstab2020cnt = pd.DataFrame(df.groupby(var_list).min_travel_2020_recode.value_counts())
    crosstab2020cnt.columns = ['voter_count']
    crosstab2020cnt = crosstab2020cnt.reset_index()
    
    var_list.append('min_travel_2020_recode')
    newvarlist2020 = var_list.copy()
    var_list.remove('min_travel_2020_recode')
    crosstab2020merge = pd.merge(crosstab2020pct,crosstab2020cnt, on = newvarlist2020, how = 'outer')
    crosstab2020merge = crosstab2020merge.rename(columns = {'min_travel_2020_recode':'travel_time',
                                                           'pct_of_group':'pct_of_group_2020',
                                                           'voter_count':'voter_count_2020'})
    
    crosstab2022pct = pd.DataFrame(df.groupby(var_list).min_travel_2022_recode.value_counts()/df.groupby(var_list).REGISTRATION_NUMBER.count())
    crosstab2022pct.columns = ['pct_of_group']
    crosstab2022pct = crosstab2022pct.reset_index()

    crosstab2022cnt = pd.DataFrame(df.groupby(var_list).min_travel_2022_recode.value_counts())
    crosstab2022cnt.columns = ['voter_count']
    crosstab2022cnt = crosstab2022cnt.reset_index()

    var_list.append('min_travel_2022_recode')
    newvarlist2022 = var_list.copy()
    var_list.remove('min_travel_2022_recode')
    crosstab2022merge = pd.merge(crosstab2022pct,crosstab2022cnt, on = newvarlist2022, how = 'outer')
    crosstab2022merge = crosstab2022merge.rename(columns = {'min_travel_2022_recode':'travel_time',
                                                           'pct_of_group':'pct_of_group_2022',
                                                           'voter_count':'voter_count_2022'})
    
    var_list.append('travel_time')
    mergevarlist = var_list.copy()
    crosstabmerge = pd.merge(crosstab2020merge, crosstab2022merge, on = mergevarlist, how = 'outer')
    crosstabmerge['pct_diff'] = crosstabmerge.pct_of_group_2020 - crosstabmerge.pct_of_group_2022
    return crosstabmerge

In [93]:
traveltimemaster5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7528561 entries, 0 to 7528560
Data columns (total 41 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0               int64  
 1   Unnamed: 0_x             int64  
 2   REGISTRATION_NUMBER      int64  
 3   BIRTHDATE_x              int64  
 4   RACE                     object 
 5   st_address               object 
 6   RESIDENCE_CITY           object 
 7   RESIDENCE_STATE          object 
 8   ZIP5                     int64  
 9   lat                      float64
 10  long                     float64
 11  min_driving_2020         float64
 12  min_driving_2022         float64
 13  driving_band_diff        float64
 14  BIRTHDATE_y              int64  
 15  min_transit_2020         float64
 16  min_transit_2022         float64
 17  transit_band_diff        float64
 18  urban                    bool   
 19  TRACTCE                  float64
 20  GEOID_left               float64
 21  NAME    

In [94]:
crosstab_traveltime_df(traveltimemaster5[traveltimemaster5.county_fips == 13121],['no_car']).sort_values(['no_car','travel_time'])

,no_car,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
0,0,600.0,0.938572,545165.0,0.506292,294077,0.432281
1,0,1200.0,0.061428,35680.0,0.483878,281058,-0.422450
4,0,1800.0,NaN,NaN,0.009831,5710,NaN
2,1,600.0,0.993156,240146.0,0.353609,85503,0.639547
3,1,1200.0,0.006844,1655.0,0.640097,154776,-0.633252
5,1,1800.0,NaN,NaN,0.006294,1522,NaN


In [95]:
votercount = pd.DataFrame(traveltimemaster5.county_fips.value_counts()/len(traveltimemaster5))

In [96]:
votercount.county_fips.iloc[:4].sum()

0.33611562687743385

In [97]:
nocarbycty =pd.DataFrame(traveltimemaster5[traveltimemaster5.no_car == True].county_fips.value_counts()/len(traveltimemaster5[traveltimemaster5.no_car == True]))

In [98]:
nocarbycty.head()

,county_fips
13121.0,0.320673
13089.0,0.106128
13051.0,0.054416
13021.0,0.045671
13245.0,0.039870


Half of the drop box ballots were returned in just four metro Atlanta counties - 
Fulton, Cobb, DeKalb, Gwinnett

In [99]:
metrocty = ['13121','13067','13089','13135']

Counting number of drop boxes by county

In [100]:
allboxes = pd.read_csv('20220520_jan_21_jan_21_may_22_unified_edited.csv')

In [101]:
allboxes.head()

,Unnamed: 0,address_may_22,lat,long,in_2022,address_jan_21,in_2021,nov_2020_form_address,in_nov_2020
0,0,"5238 Evitt St. Ringgold, GA 30736",34.915487,-85.106602,True,"5238 Evitt St. Ringgold, GA 30736",True,"5238 Evitt St. Ringgold, GA 30736",True
1,1,"1201 Sawnee Drive, Cumming, GA 30040",34.223824,-84.139148,True,"1201 Sawnee Drive, Cumming, GA 30040",True,"1201 Sawnee Drive, Cumming, GA 30040",True
2,2,"2808 N Oak St, Valdosta, GA 31602",30.869999,-83.292707,True,"2808 N Oak St, Valdosta, GA 31602",True,"2808 N Oak St, Valdosta, GA 31602",True
3,3,"56 Short Street, Dahlonega, GA 30533",34.535023,-83.981607,True,"56 Short Street, Dahlonega, GA 30533",True,"56 Short Street, Dahlonega, GA 30533",True
4,4,"167 Baker Place, Newton, GA 39870",31.317829,-84.342261,True,"167 Baker Place, Newton, GA 39870",True,"167 Baker Place, Newton, GA 39870",True


In [102]:
allboxes['geometry'] = gpd.points_from_xy(allboxes.long, allboxes.lat, crs="EPSG:4326")

In [103]:
allboxes = gpd.GeoDataFrame(allboxes)

In [104]:
# join county maps
county = gpd.read_file('tl_2020_us_county/tl_2020_us_county.shp')

In [105]:
county.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477645345,10690204,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680976231,61568965,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016818946,29090018,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169272970,22847034,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [106]:
GActy = county[county.STATEFP == '13'].copy()

In [107]:
len(GActy)

159

In [108]:
GActy = GActy.to_crs('EPSG:4326')

In [109]:
allboxes1 = gpd.sjoin(allboxes, GActy, how = 'left')

In [110]:
len(allboxes1[(allboxes1.in_nov_2020 == True) & (allboxes1.GEOID.isin(metrocty))])

106

In [111]:
len(allboxes1[(allboxes1.in_2022 == True) & (allboxes1.GEOID.isin(metrocty))])

25

In [112]:
len(allboxes1[(allboxes1.in_nov_2020 == True) & (allboxes1.GEOID == '13121')])

37

In [113]:
metrocty1 = [float(x) for x in metrocty]

In [114]:
metrocty1

[13121.0, 13067.0, 13089.0, 13135.0]

Factcheck: The changes include restricting access to dropboxes in counties that use them the most, which also have the highest number of voters of color

In [115]:
traveltimemaster5[traveltimemaster5.white == 0].county_fips.value_counts()

13121.0    115222
13135.0     85648
13089.0     68285
13067.0     62459
13117.0     34446
            ...  
13239.0       104
13061.0        99
13307.0        66
13265.0        63
13189.0        21
Name: county_fips, Length: 159, dtype: int64

Counties, by percentage of non-white voters

In [116]:
whitepctcty = pd.DataFrame(traveltimemaster5[traveltimemaster5.white == 0].groupby('county_fips').REGISTRATION_NUMBER.count()/traveltimemaster5.groupby('county_fips').REGISTRATION_NUMBER.count())

In [117]:
whitepctcty.sort_values('REGISTRATION_NUMBER', ascending = False)

,REGISTRATION_NUMBER
county_fips,
13117.0,0.203364
13039.0,0.192829
13053.0,0.156116
13179.0,0.150121
13135.0,0.140879
...,...
13307.0,0.038801
13201.0,0.034714
13075.0,0.034047


In [118]:
print('what percentage of Cobb, De Kalb, Fulton and Gwinnett voters are non-white') 
len(traveltimemaster5[(traveltimemaster5.county_fips.isin(metrocty)) & (traveltimemaster5.white == -1)])/len(traveltimemaster5[(traveltimemaster5.county_fips.isin(metrocty))])

what percentage of Cobb, De Kalb, Fulton and Gwinnett voters are non-white


0.47047995488579775

In [119]:
print('what percentage of Cobb, De Kalb, Fulton and Gwinnett voters are Black or Hispanic') 
len(traveltimemaster5[(traveltimemaster5.county_fips.isin(metrocty)) & ((traveltimemaster5.RACE == 'BH') | (traveltimemaster5.RACE == 'HP'))])/len(traveltimemaster5[(traveltimemaster5.county_fips.isin(metrocty))])

what percentage of Cobb, De Kalb, Fulton and Gwinnett voters are Black or Hispanic


0.41573432888079553

In [120]:
def time_diff_recode(x,y):
    if (pd.isna(x) == True) & (pd.isna(y) == True):
        return np.nan
    if (pd.isna(x) == True) & (pd.isna(y) == False):
        return (1)
    if (pd.isna(x) == False) & (pd.isna(y) == True):
        return(-1)
    if (pd.isna(x) == False) & (pd.isna(y) == False):
        return(x-y)

In [121]:
traveltimemaster5['travel_diff_recode'] = [time_diff_recode(x,y) for (x,y) in zip(traveltimemaster5.min_travel_2020_recode,traveltimemaster5.min_travel_2022_recode) ]

In [122]:
print('number of voters see their time go up or lose access')
len(traveltimemaster5[
                 (traveltimemaster5.travel_diff_recode < 0) ])

number of voters see their time go up or lose access


1861899

In [123]:
print ('percentage of non-white voters for voters whose travel time went up')
len(traveltimemaster5[(traveltimemaster5.travel_diff_recode < 0) & (traveltimemaster5.white == -1)])/len(traveltimemaster5[(traveltimemaster5.travel_diff_recode < 0)])

percentage of non-white voters for voters whose travel time went up


0.4511587363224321

In [124]:
print ('percentage of Black and Hispanic voters for voters whose travel time went up')
len(traveltimemaster5[(traveltimemaster5.travel_diff_recode < 0) & ((traveltimemaster5.RACE == 'BH') | (traveltimemaster5.RACE == 'HP') )])/len(traveltimemaster5[(traveltimemaster5.travel_diff_recode < 0)])

percentage of Black and Hispanic voters for voters whose travel time went up


0.4094454103042109

calculating active voter per drop box

In [125]:
voterstatus = pd.read_csv('20220715_voter_status.csv')

In [126]:
traveltimemaster6 = pd.merge(traveltimemaster5, voterstatus, on = 'REGISTRATION_NUMBER', how = 'left')

In [127]:
voterbycty = pd.DataFrame(traveltimemaster6[traveltimemaster6.VOTER_STATUS == 'A'].groupby('county_fips').REGISTRATION_NUMBER.count())

In [128]:
voterbycty.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 159 entries, 13001.0 to 13321.0
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   REGISTRATION_NUMBER  159 non-null    int64
dtypes: int64(1)
memory usage: 2.5 KB


In [129]:
voterbycty =voterbycty.reset_index()

In [130]:
voterbycty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   county_fips          159 non-null    float64
 1   REGISTRATION_NUMBER  159 non-null    int64  
dtypes: float64(1), int64(1)
memory usage: 2.6 KB


In [131]:
dropboxbycty22 = pd.DataFrame(allboxes1[allboxes1.in_2022 == True].groupby('GEOID').lat.count())
dropboxbycty22 = dropboxbycty22.reset_index()
dropboxbycty22.GEOID = dropboxbycty22.GEOID.astype('float')

In [132]:
boxpervoter22 = pd.merge(voterbycty,dropboxbycty22, left_on = 'county_fips', right_on = 'GEOID', how = 'outer')

In [133]:
boxpervoter22['box_per_100k_voter_2022'] = boxpervoter22.lat/boxpervoter22.REGISTRATION_NUMBER*100000

In [134]:
boxpervoter22 = boxpervoter22.rename(columns = {'lat':'box_count'})

In [135]:
ctynames = pd.read_excel('county_fips.xlsx')

In [136]:
boxpervoter22_1 = pd.merge(boxpervoter22, ctynames, left_on = 'county_fips', right_on = 'FIPS', how = 'left')

In [137]:
allboxes1.head()

,Unnamed: 0,address_may_22,lat,long,in_2022,address_jan_21,in_2021,nov_2020_form_address,in_nov_2020,geometry,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
0,0,"5238 Evitt St. Ringgold, GA 30736",34.915487,-85.106602,True,"5238 Evitt St. Ringgold, GA 30736",True,"5238 Evitt St. Ringgold, GA 30736",True,POINT (-85.10660 34.91549),...,H1,G4020,174,16860,None,A,419998923,771159,+34.9002118,-085.1393851
1,1,"1201 Sawnee Drive, Cumming, GA 30040",34.223824,-84.139148,True,"1201 Sawnee Drive, Cumming, GA 30040",True,"1201 Sawnee Drive, Cumming, GA 30040",True,POINT (-84.13915 34.22382),...,H1,G4020,122,12060,None,A,581757645,58447999,+34.2251430,-084.1274075
2,2,"2808 N Oak St, Valdosta, GA 31602",30.869999,-83.292707,True,"2808 N Oak St, Valdosta, GA 31602",True,"2808 N Oak St, Valdosta, GA 31602",True,POINT (-83.29271 30.87000),...,H1,G4020,None,46660,None,A,1287842038,34542105,+30.8331450,-083.2689952
3,3,"56 Short Street, Dahlonega, GA 30533",34.535023,-83.981607,True,"56 Short Street, Dahlonega, GA 30533",True,"56 Short Street, Dahlonega, GA 30533",True,POINT (-83.98161 34.53502),...,H1,G4020,None,None,None,A,732820607,3275562,+34.5681403,-083.9989251
4,4,"167 Baker Place, Newton, GA 39870",31.317829,-84.342261,True,"167 Baker Place, Newton, GA 39870",True,"167 Baker Place, Newton, GA 39870",True,POINT (-84.34226 31.31783),...,H1,G4020,None,None,None,A,885665381,18598653,+31.3195798,-084.4548812


In [138]:
dropboxbycty20 = pd.DataFrame(allboxes1[allboxes1.in_nov_2020 == True].groupby('GEOID').lat.count())
dropboxbycty20 = dropboxbycty20.reset_index()
dropboxbycty20.GEOID = dropboxbycty20.GEOID.astype('float')

In [139]:
boxpervoter20 = pd.merge(voterbycty,dropboxbycty20, left_on = 'county_fips', right_on = 'GEOID', how = 'outer')

In [140]:
boxpervoter20['box_per_100k_voter_2020'] = boxpervoter20.lat/boxpervoter20.REGISTRATION_NUMBER*100000

In [141]:
boxpervoter20 = boxpervoter20.rename(columns = {'lat':'box_count'})

In [142]:
ctynames = pd.read_excel('county_fips.xlsx')

In [143]:
boxpervoter20_1 = pd.merge(boxpervoter20, ctynames, left_on = 'county_fips', right_on = 'FIPS', how = 'left')

In [144]:
boxpervoter20_1.to_csv('20220715_box_per_100k_active_voter_2020.csv')

In [145]:
voterctyrace = pd.DataFrame(traveltimemaster6[(traveltimemaster6.VOTER_STATUS == 'A') & (traveltimemaster6.white == -1)].groupby('county_fips').REGISTRATION_NUMBER.count()/traveltimemaster6[traveltimemaster6.VOTER_STATUS == 'A'].groupby('county_fips').REGISTRATION_NUMBER.count())

In [146]:
traveltimemaster6[(traveltimemaster6.VOTER_STATUS == 'A') & (traveltimemaster6.white == -1)].groupby('county_fips').REGISTRATION_NUMBER.count()

county_fips
13001.0     2220
13003.0     1160
13005.0      889
13007.0      865
13009.0    10646
           ...  
13313.0    11437
13315.0     1156
13317.0     2447
13319.0     2354
13321.0     3381
Name: REGISTRATION_NUMBER, Length: 159, dtype: int64

limitations on drop boxes are making it more difficult for those communities that rely on them most.

In [147]:
pd.DataFrame(traveltimemaster6.groupby('county_fips').REGISTRATION_NUMBER.count()).sort_values('REGISTRATION_NUMBER',ascending = False)

,REGISTRATION_NUMBER
county_fips,
13121.0,822646
13135.0,607955
13089.0,553528
13067.0,546338
13051.0,221442
...,...
13061.0,2157
13125.0,2015
13239.0,1726


In [148]:
metrocty

['13121', '13067', '13089', '13135']

the metro counties are the four largest counties

Owens is one of at least 1.9 million Georgians – or a QUARTER of all voters – who saw their travel time to a drop box increase under the new law. More than 90 percent of those voters live in cities or suburbs, home to most of the state’s voters of color. 

In [149]:
traveltimemaster6[traveltimemaster6.travel_diff_recode<0].urban.value_counts()

True     1702711
False     159188
Name: urban, dtype: int64

In [150]:
1702711/(1702711+159188)

0.9145023441121135

In [151]:
traveltimemaster6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7528561 entries, 0 to 7528560
Data columns (total 44 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0_x             int64  
 1   Unnamed: 0_x             int64  
 2   REGISTRATION_NUMBER      int64  
 3   BIRTHDATE_x              int64  
 4   RACE                     object 
 5   st_address               object 
 6   RESIDENCE_CITY           object 
 7   RESIDENCE_STATE          object 
 8   ZIP5                     int64  
 9   lat                      float64
 10  long                     float64
 11  min_driving_2020         float64
 12  min_driving_2022         float64
 13  driving_band_diff        float64
 14  BIRTHDATE_y              int64  
 15  min_transit_2020         float64
 16  min_transit_2022         float64
 17  transit_band_diff        float64
 18  urban                    bool   
 19  TRACTCE                  float64
 20  GEOID_left               float64
 21  NAME    

In [152]:
fivemin = pd.read_csv('ORIGINAL__20220627_voter_data_all_vars.csv') 

In [153]:
fivemin.min_travel_time_2020.value_counts()

600.0     3014662
300.0     1498940
900.0     1318056
1200.0     689748
1500.0     262923
1800.0      78501
Name: min_travel_time_2020, dtype: int64

In [154]:
fivemin.travel_diff.value_counts()

 0.0       4315789
-300.0     1568247
-600.0      682819
 1.0        627759
-900.0      163011
 300.0       82922
-1.0         31666
-1200.0      14781
 600.0        2992
 900.0         518
-1500.0         85
Name: travel_diff, dtype: int64

In [155]:
fivemin[(fivemin.travel_diff<= -600) | (fivemin.travel_diff == -1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892362 entries, 28 to 7528532
Data columns (total 40 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               892362 non-null  int64  
 1   Unnamed: 0_x             892362 non-null  int64  
 2   REGISTRATION_NUMBER      892362 non-null  int64  
 3   BIRTHDATE_x              892362 non-null  int64  
 4   RACE                     892362 non-null  object 
 5   st_address               892362 non-null  object 
 6   RESIDENCE_CITY           892362 non-null  object 
 7   RESIDENCE_STATE          892362 non-null  object 
 8   ZIP5                     892362 non-null  int64  
 9   lat                      892362 non-null  float64
 10  long                     892362 non-null  float64
 11  min_driving_2020         892362 non-null  float64
 12  min_driving_2022         860696 non-null  float64
 13  driving_band_diff        860696 non-null  float64
 14  BI

In [156]:
fivemin[(fivemin.travel_diff< -300) | (fivemin.travel_diff == -1)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892362 entries, 28 to 7528532
Data columns (total 40 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               892362 non-null  int64  
 1   Unnamed: 0_x             892362 non-null  int64  
 2   REGISTRATION_NUMBER      892362 non-null  int64  
 3   BIRTHDATE_x              892362 non-null  int64  
 4   RACE                     892362 non-null  object 
 5   st_address               892362 non-null  object 
 6   RESIDENCE_CITY           892362 non-null  object 
 7   RESIDENCE_STATE          892362 non-null  object 
 8   ZIP5                     892362 non-null  int64  
 9   lat                      892362 non-null  float64
 10  long                     892362 non-null  float64
 11  min_driving_2020         892362 non-null  float64
 12  min_driving_2022         860696 non-null  float64
 13  driving_band_diff        860696 non-null  float64
 14  BI

In [157]:
fivemin[(fivemin.travel_diff< 0)].travel_diff.value_counts()

-300.0     1568247
-600.0      682819
-900.0      163011
-1.0         31666
-1200.0      14781
-1500.0         85
Name: travel_diff, dtype: int64

In [158]:
fivemin.min_travel_time_2020.value_counts()

600.0     3014662
300.0     1498940
900.0     1318056
1200.0     689748
1500.0     262923
1800.0      78501
Name: min_travel_time_2020, dtype: int64

In [159]:
fivemin.groupby(['min_travel_time_2020','min_travel_time_2022']).REGISTRATION_NUMBER.count()

min_travel_time_2020  min_travel_time_2022
300.0                 300.0                    559958
                      600.0                    417719
                      900.0                    394347
                      1200.0                   111395
                      1500.0                    13655
                      1800.0                       85
600.0                 300.0                     38224
                      600.0                   1743351
                      900.0                    945580
                      1200.0                   237398
                      1500.0                    43963
                      1800.0                     1126
900.0                 300.0                      1074
                      600.0                     24641
                      900.0                   1064654
                      1200.0                   168594
                      1500.0                    44458
                      1800.0           

In [160]:
traveltimemaster6[traveltimemaster6.travel_diff_recode<0].travel_diff_recode.value_counts()

-600.0     1770321
-1200.0      59654
-1.0         31924
Name: travel_diff_recode, dtype: int64

# Fact checks

##### "The changes include restricting access to drop boxes in counties that used them the most"

According to ballot transfer forms compiled by GPB, WABE and NPR, Cobb, DeKalb, Fulton and Gwinnett has the most number of ballot by drop box. Their fips codes are 13121, 13067, 13089, and 13135

In [161]:
ctyct = crosstab_traveltime(['county_fips'])

In [162]:
ctyct.sort_values('pct_diff', ascending = False).head(20)

,county_fips,travel_time,pct_of_group_2020,voter_count_2020,pct_of_group_2022,voter_count_2022,pct_diff
276,13245.0,600.0,0.765832,113105.0,0.190224,28094,0.575608
181,13163.0,600.0,0.845354,9588.0,0.295715,3354,0.549639
132,13121.0,600.0,0.954616,785311.0,0.461414,379580,0.493202
146,13135.0,600.0,0.863163,524764.0,0.394342,239742,0.468821
73,13067.0,600.0,0.938253,512603.0,0.488128,266683,0.450124
104,13097.0,600.0,0.748658,78097.0,0.300357,31332,0.448301
39,13029.0,1200.0,0.575887,19006.0,0.157834,5209,0.418053
279,13247.0,600.0,0.974431,65167.0,0.573441,38350,0.400990
120,13113.0,600.0,0.718119,67236.0,0.320065,29967,0.398054
66,13059.0,600.0,0.885659,70378.0,0.500919,39805,0.384740


We can see that in these four counties, the percentage of voters who could access a drop box within 10 minutes were around 90% in 2020 and all fell by at least 30 percentage points in 2022.

##### "...counties that used them the most, which also have the highest number of voters of color "

In [163]:
metrocty

['13121', '13067', '13089', '13135']

In [164]:
voterofcolor = pd.DataFrame(traveltimemaster6[(traveltimemaster6.white == -1)].groupby('county_fips').REGISTRATION_NUMBER.count())

In [165]:
voterofcolor.sort_values('REGISTRATION_NUMBER', ascending = False).head(5)

,REGISTRATION_NUMBER
county_fips,
13121.0,395820
13089.0,310448
13135.0,290153
13067.0,194113
13063.0,153020


##### "calculating travel time intervals to a drop box for over  7.5 million voters."

In [166]:
len(traveltimemaster6)

7528561

##### "Cobb, DeKalb, Fulton and Gwinnett–where about 50% of the voters are people of color. "

In [167]:
print ('Total number of voters of color in Cobb, DeKalb, Fulton and Gwinnett')
pocvotermetro4 = len(traveltimemaster6[(traveltimemaster6.white == -1) & (traveltimemaster6.county_fips.isin(metrocty))])
print(pocvotermetro4)

Total number of voters of color in Cobb, DeKalb, Fulton and Gwinnett
1190534


In [168]:
print ('Total number of voters in Cobb, DeKalb, Fulton and Gwinnett')
totalvotermetro4 = len(traveltimemaster6[(traveltimemaster6.county_fips.isin(metrocty))])
print(totalvotermetro4)

Total number of voters in Cobb, DeKalb, Fulton and Gwinnett
2530467


In [169]:
print('Percentage of voters of color in Cobb, DeKalb, Fulton and Gwinnett')
pocvotermetro4/totalvotermetro4

Percentage of voters of color in Cobb, DeKalb, Fulton and Gwinnett


0.47047995488579775

##### "Under the new law, the number of drop boxes in these four counties plummeted from 106 to 25"

In [170]:
print('Number of drop boxes in Cobb, DeKalb, Fulton and Gwinnett, 2020')
len(allboxes1[(allboxes1.in_nov_2020 == True) & (allboxes1.GEOID.isin(metrocty))])

Number of drop boxes in Cobb, DeKalb, Fulton and Gwinnett, 2020


106

In [171]:
print('Number of drop boxes in Cobb, DeKalb, Fulton and Gwinnett, 2022')
len(allboxes1[(allboxes1.in_2022 == True) & (allboxes1.GEOID.isin(metrocty))])

Number of drop boxes in Cobb, DeKalb, Fulton and Gwinnett, 2022


25

##### "About 1.9 million people, a quarter of the state’s voters, have seen their travel time to a drop box increase from the 2020 election."

In [172]:
timegoup = len(traveltimemaster6[traveltimemaster6.travel_diff_recode<0])
print(timegoup)

1861899


##### "More than 90% of voters who saw an increase in their travel time to a drop box live in cities or suburbs"

In [173]:
print('Number of voters of color who see increase in travel time and live in urban areas')
urbantimegoup = len(traveltimemaster6[(traveltimemaster6.travel_diff_recode<0) & (traveltimemaster6.urban == 1)])
print (urbantimegoup)

Number of voters of color who see increase in travel time and live in urban areas
1702711


In [174]:
urbantimegoup/timegoup

0.9145023441121135

 ##### "live in cities or suburbs, which are home to most of the state’s minority voters"

In [175]:
print ('Number of voters of color who live in urban areas')
pocurban = len(traveltimemaster6[(traveltimemaster6.urban == 1) & (traveltimemaster6.white == -1)])
print (pocurban)

Number of voters of color who live in urban areas
2403485


In [176]:
print ('Number of voters of color')
poccount = len(traveltimemaster6[(traveltimemaster6.white == -1)])
print (poccount)

Number of voters of color
2738705


In [177]:
print ('Percentage of voters of color who live in urban areas')
pocurban/poccount

Percentage of voters of color who live in urban areas


0.8775990842387187

##### "...vote heavily demoratic"

In [178]:
traveltimemaster6[(traveltimemaster6.urban == 1)].party.value_counts()

D      3441848
R      2139193
tie       5275
Name: party, dtype: int64

##### "Twenty-three drop boxes were spread across Gwinnett County in 2020. Under the new law, only five remain"

In [179]:
len(allboxes1[(allboxes1.in_nov_2020 == 1) & (allboxes1.GEOID == '13135')])

23

In [180]:
len(allboxes1[(allboxes1.in_2022 == 1) & (allboxes1.GEOID == '13135')])

6

##### "In 2020, more than 70% of urban and suburban voters lived within 10 minutes’ travel of a drop box, the analysis by NPR, WABE and GPB found. That number dropped to less than 50% for this year’s midterms. "

In [181]:
print ("Percentage of voters in urban areas that live within 10 minutes of a drop box, 2020")
len(traveltimemaster6[(traveltimemaster6.urban == 1) & (traveltimemaster6.min_travel_2020_recode == 600)])/len(traveltimemaster6[(traveltimemaster6.urban == 1)])

Percentage of voters in urban areas that live within 10 minutes of a drop box, 2020


0.7343953647437769

In [182]:
print ("Percentage of voters in urban areas that live within 10 minutes of a drop box, 2022")
len(traveltimemaster6[(traveltimemaster6.urban == 1) & (traveltimemaster6.min_travel_2022_recode == 600)])/len(traveltimemaster6[(traveltimemaster6.urban == 1)])

Percentage of voters in urban areas that live within 10 minutes of a drop box, 2022


0.468518998188637

##### "The percentage of rural voters who can access a drop box within 10 minutes of their home this year is nearly 20%"

In [183]:
print ("Percentage of voters not in urban areas that live within 10 minutes of a drop box, 2020")
len(traveltimemaster6[(traveltimemaster6.urban == 0) & (traveltimemaster6.min_travel_2020_recode == 600)])/len(traveltimemaster6[(traveltimemaster6.urban == 0)])

Percentage of voters not in urban areas that live within 10 minutes of a drop box, 2020


0.2190427153272645

In [184]:
print ("Percentage of voters not in urban areas that live within 10 minutes of a drop box, 2022")
len(traveltimemaster6[(traveltimemaster6.urban == 0) & (traveltimemaster6.min_travel_2022_recode == 600)])/len(traveltimemaster6[(traveltimemaster6.urban == 0)])

Percentage of voters not in urban areas that live within 10 minutes of a drop box, 2022


0.21957974626189397

##### "election officials were forced to slash the county’s drop box offerings from 38 to seven. "

In [185]:
len(allboxes1[(allboxes1.in_nov_2020 == 1) & (allboxes1.GEOID == '13121')])

37

In [186]:
(allboxes1[(allboxes1.in_nov_2020 == 1) & (allboxes1.GEOID == '13067')])

,Unnamed: 0,address_may_22,lat,long,in_2022,address_jan_21,in_2021,nov_2020_form_address,in_nov_2020,geometry,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
5,5,"4400 Lower Roswell Rd, Marietta, GA 30068",33.963188,-84.423463,True,"4400 Lower Roswell Rd, Marietta, GA 30068",True,"4400 Lower Roswell Rd, Marietta, GA 30068",True,POINT (-84.42346 33.96319),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
63,64,"736 Whitlock Avenue, Suite 400, Marietta, GA 3...",33.953518,-84.573838,True,"736 Whitlock Avenue, Marietta GA",True,"736 Whitlock Avenue, Suite 400, Marietta, GA 3...",True,POINT (-84.57384 33.95352),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
66,68,"805 Clay Rd, Mableton GA 30126",33.825185,-84.577601,True,"805 Clay St. Mableton, GA",True,"805 Clay Rd, Mableton GA 30126",True,POINT (-84.57760 33.82518),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
67,70,"1750 Dennis Kemp Ln NW, Kennesaw GA 30152",33.999632,-84.701901,True,"1750 Dennis Kemp Ln NW Kennesaw, GA",True,"1750 Dennis Kemp Ln NW, Kennesaw GA 30152",True,POINT (-84.70190 33.99963),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
202,207,NaN,34.025209,-84.524229,NaN,880 Shaw Park Rd. Marietta,True,"880 Shaw Park Road, Marietta",True,POINT (-84.52423 34.02521),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
203,208,NaN,33.857269,-84.675738,NaN,4181 Atlanta St. Powder Springs,True,"4181 Atlanta Street, Powder Springs",True,POINT (-84.67574 33.85727),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
204,209,NaN,33.777036,-84.580525,NaN,South Cobb Rec Center 875 Riverside Pkwy Auste...,True,"875 Riverside Pkwy, Austell",True,POINT (-84.58052 33.77704),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
294,301,NaN,34.008879,-84.606325,NaN,"2380 N. Cobb Pkwy Kennesaw, GA",True,"2380 N. Cobb Pkwy, Kennesaw",True,POINT (-84.60632 34.00888),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
295,302,NaN,34.028897,-84.469754,NaN,3320 Sandy Plains Rd. Marietta,True,"3320 Sandy Plains Road, Marietta",True,POINT (-84.46975 34.02890),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213
296,303,NaN,33.840410,-84.610998,NaN,"4700 Austell Rd, Austell, GA 30106",True,"4700 Austell Rd, Austell",True,POINT (-84.61100 33.84041),...,H1,G4020,122,12060,None,A,880026860,12347127,+33.9399286,-084.5741213


In [187]:
len(allboxes1[(allboxes1.in_2022 == 1) & (allboxes1.GEOID == '13121')])

7

##### "In 2020, at least 90% of voters in those communities with the lowest rates of vehicle ownership were able to reach a drop box within 10 minutes. By 2022, that plummeted to 60%"

In [188]:
print("Percentage of communities with the lowest rates of vehicle ownership within 10 minutes' travel of drop box, 2020 ")
len(traveltimemaster6[(traveltimemaster6.no_car == 1) & (traveltimemaster6.min_travel_2020_recode == 600)])/len(traveltimemaster6[(traveltimemaster6.no_car == 1)])

Percentage of communities with the lowest rates of vehicle ownership within 10 minutes' travel of drop box, 2020 


0.8814245341446045

In [189]:
print("Percentage of communities with the lowest rates of vehicle ownership within 10 minutes' travel of drop box, 2022 ")
len(traveltimemaster6[(traveltimemaster6.no_car == 1) & (traveltimemaster6.min_travel_2022_recode == 600)])/len(traveltimemaster6[(traveltimemaster6.no_car == 1)])

Percentage of communities with the lowest rates of vehicle ownership within 10 minutes' travel of drop box, 2022 


0.5612305929502694

##### "These neighborhoods have some of the lowest median household income in the state,"

In [190]:
traveltimemaster6[(traveltimemaster6.no_car == 1)].groupby('TRACTCE').income_qt.first().value_counts()

1.0    235
2.0     37
3.0     25
4.0      9
Name: income_qt, dtype: int64

##### The bulk are majority Black

In [191]:
blackpct = pd.DataFrame(traveltimemaster6[(traveltimemaster6.no_car == 1) & (traveltimemaster6.RACE == 'BH')].groupby('TRACTCE').REGISTRATION_NUMBER.count()/traveltimemaster6[(traveltimemaster6.no_car == 1) ].groupby('TRACTCE').REGISTRATION_NUMBER.count())

In [192]:
blackpct.columns = ['black_pct']

In [193]:
print("Percent of the census tracts with the lowest levels of vehicle ownership whose voters are majority Black")
len(blackpct[blackpct.black_pct>=.5])/len(blackpct)

Percent of the census tracts with the lowest levels of vehicle ownership whose voters are majority Black


0.6134185303514377

##### voted overwhelmingly for Biden.

In [194]:
partybytract = pd.DataFrame(traveltimemaster6[(traveltimemaster6.no_car == 1)].groupby('TRACTCE').party.first())

In [195]:
partybytract.party.value_counts()

D      254
R       58
tie      1
Name: party, dtype: int64

##### Heard county voter count

In [196]:
traveltimemaster6[traveltimemaster6.county_fips == 13149].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8122 entries, 7229 to 7525438
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0_x             8122 non-null   int64  
 1   Unnamed: 0_x             8122 non-null   int64  
 2   REGISTRATION_NUMBER      8122 non-null   int64  
 3   BIRTHDATE_x              8122 non-null   int64  
 4   RACE                     8122 non-null   object 
 5   st_address               8122 non-null   object 
 6   RESIDENCE_CITY           8122 non-null   object 
 7   RESIDENCE_STATE          8122 non-null   object 
 8   ZIP5                     8122 non-null   int64  
 9   lat                      8122 non-null   float64
 10  long                     8122 non-null   float64
 11  min_driving_2020         0 non-null      float64
 12  min_driving_2022         8122 non-null   float64
 13  driving_band_diff        0 non-null      float64
 14  BIRTHDATE_y       